In [13]:
from myUtils import *
import pandas as pd

In [14]:
# Validates that putting together all the dataset ids on every day of a week are the same as the
# dataset ids of the week
def validate_days_in_weeks(week_ts, weeks_ds, days_ds):
    
    ret_val = True
    # Get the set of dataset_ids accessed in the week identified by 'week_ts'
    week_set = set(weeks_ds[weeks_ds['week_ts']==week_ts].datasets_set.values[0])    
    
    # Get the set of dataset_ids accessed in every day that belongs to a week identified by 'week_ts'
    days_set=set()
    for day_set in days_ds[days_ds['week_ts']==week_ts]['datasets_set']:
        days_set.update(set(day_set))
    
    # Makes sure both sets are the same size
    week_set_len= len(week_set)
    days_set_len= len(days_set)
    if week_set_len != days_set_len:
        print(str(week_set_len)+" != "+str(days_set_len))
        ret_val = False
    
    # If both sets are the same size proceed to make sure tha both
    # sets contain the same items
    if ret_val != False:
        if days_set != week_set:
            ret_val = False
    
    return ret_val

In [18]:
# the code below validates that the spliting of job records into different data tier classes
# is correct by comparing the size of the sizes of the set of datasets accessed in a specific day for:
# 1. The DataFrame where all datatiers are together and
# 2. The DataFrame where datatiers asre separated in 4 classes (0 to 3)

datasets_df = pd.read_parquet("/Users/ddavila/projects/DOMA/data/model/dataset.parquet/")
datasets_size = datasets_df[['d_dataset_id', 'dataset_size']]
# Read DatafRame where job records are separated in 4 datatier classes
datasets_tiers = pd.read_parquet("/Users/ddavila/projects/DOMA/data/model/data_tier_days_201906.parquet/")

# Read DatafRame where jobs from different datatiers are put together
datasets_all = pd.read_parquet("/Users/ddavila/projects/DOMA/data/model/days_201906.parquet/")

# Claculate the size of the DataFrame with no datatier distinction
all_set=set()
for i in datasets_all[datasets_all['day_ts']==1559779200.0].datasets_set.values[0]:
    all_set.add(i)
all_size = get_size_of_datasets_set(all_set, datasets_size)    
print(format_bytes(all_size))

# Calculate the size of each datatier class
tier_set0=set()
a = datasets_tiers['data_data_tier_name_class']==0
b = datasets_tiers['day_ts']==1559779200.0
for j in datasets_tiers[a&b].datasets_set.values[0]:
    tier_set0.add(j)
tier0_size = get_size_of_datasets_set(tier_set0, datasets_size)
print(format_bytes(tier0_size))

tier_set1=set()
a = datasets_tiers['data_data_tier_name_class']==1
b = datasets_tiers['day_ts']==1559779200.0
for j in datasets_tiers[a&b].datasets_set.values[0]:
    tier_set1.add(j)
tier1_size = get_size_of_datasets_set(tier_set1, datasets_size)
print(format_bytes(tier1_size))

tier_set2=set()
a = datasets_tiers['data_data_tier_name_class']==2
b = datasets_tiers['day_ts']==1559779200.0
for j in datasets_tiers[a&b].datasets_set.values[0]:
    tier_set2.add(j)
tier2_size = get_size_of_datasets_set(tier_set2, datasets_size)
print(format_bytes(tier2_size))

tier_set3=set()
a = datasets_tiers['data_data_tier_name_class']==3
b = datasets_tiers['day_ts']==1559779200.0
for j in datasets_tiers[a&b].datasets_set.values[0]:
    tier_set3.add(j)
tier3_size = get_size_of_datasets_set(tier_set3, datasets_size)
print(format_bytes(tier3_size))

# The sum of the sizes of the different classes is the same as the
# size of all the classes put together
print(tier0_size + tier1_size +tier2_size +tier3_size)
print(all_size)

5.13 PB
997.96 TB
2.20 TB
2.78 PB
1.36 PB
5134743402851260.0
5134743402851260.0
